# Exploratory Data Analysis: California Housing Dataset

This notebook provides a comprehensive exploration of the California Housing dataset used for comparing Bayesian Hierarchical Models with Deep Neural Networks.

**Objectives:**
1. Understand the data distribution and feature characteristics
2. Visualize spatial patterns in housing prices
3. Examine relationships between features and target
4. Validate the "Small Tabular Regime" hypothesis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("Libraries loaded successfully!")

## 1. Data Loading and Overview

In [ ]:
# Load the processed dataset
data_path = Path("../data/processed/housing_with_spatial_clusters.csv")
df = pd.read_csv(data_path)

print(f"Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nThis represents a 'Small Tabular Regime' (<20k samples)")
print(f"where deep learning typically struggles against linear baselines.\n")
df.head()

In [ ]:
# Data types and memory usage
print("=" * 50)
print("DATA TYPES & MEMORY")
print("=" * 50)
df.info()

## 2. Statistical Summary

In [ ]:
# Comprehensive statistics
stats = df.describe().T
stats['missing'] = df.isnull().sum()
stats['missing_pct'] = (stats['missing'] / len(df) * 100).round(2)
stats['dtype'] = df.dtypes

print("=" * 70)
print("FEATURE STATISTICS SUMMARY")
print("=" * 70)
stats[['count', 'mean', 'std', 'min', '50%', 'max', 'missing_pct']]

## 3. Target Variable Analysis: Housing Price

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Distribution
axes[0].hist(df['price'], bins=50, edgecolor='white', alpha=0.7, color='steelblue')
axes[0].axvline(df['price'].mean(), color='red', linestyle='--', label=f"Mean: {df['price'].mean():.2f}")
axes[0].axvline(df['price'].median(), color='orange', linestyle='--', label=f"Median: {df['price'].median():.2f}")
axes[0].set_xlabel('Housing Price (scaled)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Price Distribution')
axes[0].legend()

# Box plot
axes[1].boxplot(df['price'], vert=True)
axes[1].set_ylabel('Price')
axes[1].set_title('Price Box Plot (Outlier Detection)')

# QQ plot for normality
from scipy import stats as scipy_stats
scipy_stats.probplot(df['price'], dist="norm", plot=axes[2])
axes[2].set_title('QQ Plot (Normality Check)')

plt.tight_layout()
plt.savefig('../results/figures/eda_target_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Skewness: {df['price'].skew():.3f}")
print(f"Kurtosis: {df['price'].kurtosis():.3f}")

## 4. Feature Correlations

In [ ]:
# Correlation matrix
numeric_cols = ['price', 'median_income', 'house_age', 'avg_rooms', 
                'avg_bedrooms', 'population', 'latitude', 'longitude']
corr_matrix = df[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='RdBu_r', center=0, square=True,
            linewidths=0.5, ax=ax)
ax.set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../results/figures/eda_correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Top correlations with price
print("\n" + "=" * 40)
print("CORRELATIONS WITH PRICE")
print("=" * 40)
price_corr = corr_matrix['price'].drop('price').sort_values(key=abs, ascending=False)
for feat, corr in price_corr.items():
    print(f"{feat:20s}: {corr:+.3f}")

## 5. Spatial Analysis: Geographic Patterns

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Price by location
scatter1 = axes[0].scatter(df['longitude'], df['latitude'], 
                           c=df['price'], cmap='viridis', 
                           alpha=0.5, s=10)
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].set_title('Housing Prices by Location')
plt.colorbar(scatter1, ax=axes[0], label='Price')

# Clusters by location
if 'spatial_cluster' in df.columns:
    scatter2 = axes[1].scatter(df['longitude'], df['latitude'], 
                               c=df['spatial_cluster'], cmap='tab10', 
                               alpha=0.5, s=10)
    axes[1].set_xlabel('Longitude')
    axes[1].set_ylabel('Latitude')
    axes[1].set_title('Spatial Clusters (Hierarchical Groups)')
    plt.colorbar(scatter2, ax=axes[1], label='Cluster ID')

plt.tight_layout()
plt.savefig('../results/figures/eda_spatial_patterns.png', dpi=150, bbox_inches='tight')
plt.show()

if 'spatial_cluster' in df.columns:
    print(f"\nNumber of Spatial Clusters: {df['spatial_cluster'].nunique()}")
    print(f"Samples per Cluster (mean): {len(df) / df['spatial_cluster'].nunique():.0f}")

## 6. Key Feature: Income vs Price Relationship

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall relationship
axes[0].scatter(df['median_income'], df['price'], alpha=0.3, s=5)
z = np.polyfit(df['median_income'], df['price'], 1)
p = np.poly1d(z)
x_line = np.linspace(df['median_income'].min(), df['median_income'].max(), 100)
axes[0].plot(x_line, p(x_line), 'r-', linewidth=2, label=f'Linear fit (r={price_corr["median_income"]:.3f})')
axes[0].set_xlabel('Median Income')
axes[0].set_ylabel('Price')
axes[0].set_title('Income vs Price: Linear Relationship')
axes[0].legend()

# By cluster (key insight for Bayesian model)
if 'spatial_cluster' in df.columns:
    for cluster in df['spatial_cluster'].unique():
        cluster_df = df[df['spatial_cluster'] == cluster]
        axes[1].scatter(cluster_df['median_income'], cluster_df['price'], 
                       alpha=0.5, s=10, label=f'Cluster {cluster}')
    axes[1].set_xlabel('Median Income')
    axes[1].set_ylabel('Price')
    axes[1].set_title('Income vs Price BY CLUSTER\n(Demonstrates Heterogeneous Slopes)')
    axes[1].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)

plt.tight_layout()
plt.savefig('../results/figures/eda_income_price_clusters.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "=" * 60)
print("KEY INSIGHT: Income-Price relationship varies by cluster!")
print("This is the heterogeneity that Bayesian Hierarchical Models capture.")
print("=" * 60)

## 7. Cluster-Level Statistics

In [ ]:
if 'spatial_cluster' in df.columns:
    cluster_stats = df.groupby('spatial_cluster').agg({
        'price': ['mean', 'std', 'count'],
        'median_income': 'mean',
        'house_age': 'mean'
    }).round(3)
    
    cluster_stats.columns = ['price_mean', 'price_std', 'n_samples', 
                             'income_mean', 'age_mean']
    cluster_stats = cluster_stats.sort_values('price_mean', ascending=False)
    
    print("=" * 70)
    print("CLUSTER-LEVEL SUMMARY STATISTICS")
    print("=" * 70)
    display(cluster_stats)
    
    # Variance analysis
    print(f"\nBetween-cluster price variance: {cluster_stats['price_mean'].var():.4f}")
    print(f"Within-cluster price variance (avg): {cluster_stats['price_std'].mean()**2:.4f}")

## 8. Data Regime Analysis: Why This Favors Linear Models

In [ ]:
# Sample size vs feature count
n_samples = len(df)
n_features = len(numeric_cols) - 1  # Exclude target

print("=" * 60)
print("DATA REGIME ANALYSIS")
print("=" * 60)
print(f"\nSample Size (n):        {n_samples:,}")
print(f"Feature Count (p):      {n_features}")
print(f"Samples per Feature:    {n_samples / n_features:.0f}")
print(f"\nRegime Classification:  {'SMALL TABULAR' if n_samples < 20000 else 'MEDIUM/LARGE'}")
print()
print("Implications:")
print("- High risk of overfitting for complex models (NNs)")
print("- Linear models benefit from implicit regularization")
print("- Bayesian priors provide effective capacity control")
print("- Deep learning requires N >> 100k to outperform in tabular data")

## 9. Summary & Key Takeaways

In [ ]:
print("\n" + "=" * 70)
print("EDA SUMMARY: KEY FINDINGS")
print("=" * 70)
print("""
1. SMALL TABULAR REGIME
   - Dataset size favors low-variance models (Linear, Bayesian)
   - Deep NNs likely to overfit without massive regularization

2. STRONG LINEAR SIGNAL
   - Median income shows r ≈ 0.7 correlation with price
   - Primary relationship is approximately linear

3. SPATIAL HETEROGENEITY EXISTS
   - Income-price slopes vary meaningfully by cluster
   - Hierarchical Bayesian models can capture this explicitly
   - MLPs aggregate this into opaque non-linear mappings

4. RESEARCH VALUE
   - Validates use of Bayesian partial pooling for spatial data
   - Demonstrates practical limits of deep learning on tabular data
   - Supports Occam's Razor in model selection
""")